In [ ]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob
import json
from nltk.tokenize import word_tokenize
from dateutil import parser
import datetime

In [ ]:
consumer_key = 'pZnrePxPDwk3qjE5LZ6d9uucJ'
consumer_secret = 'OJYnxte3rsqrkN0TjMOsErIl1mgO9eTRNbUq0KgBsWDGWMj0r1'
access_token = '1107646065459818498-WSJziWOyGeeAwlAVjXMjLssZK2dqfr'
access_token_secret = 'QaUOE0vHW3TqGGAXfp23g5VTUAgf1ucZpzxXiI8x6Dfzp'
api=None
try:
    
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
 
    api = tweepy.API(auth)
except:
    print("Error: Authentication Failed")

In [ ]:
 def get_tweet_sentiment(tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'

In [ ]:
def fetch_data_frm_twitter(argurment,language,result_typ):
    args=[argurment]
    api= tweepy.API(auth,wait_on_rate_limit=True)
    list_tweets= [] # storing tweets in empty list
    query=args[0]
    if len(args) == 1:
        for status in tweepy.Cursor(api.search,q=query+" -filter:retweets",lang=language,result_type=result_typ).items(2000):
            list_tweets.append(status._json)
    return list_tweets

In [ ]:
Messi_tweets = fetch_data_frm_twitter('Messi','en','recent')

In [ ]:
print(len(Messi_tweets))

In [63]:
def get_tweets(query, count): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        Ten_days_ago = datetime.datetime.utcnow()-datetime.timedelta(days = 10)
        print(Ten_days_ago)
        # empty list to store parsed tweets 
        tweets = [] 
  
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = api.search(q = query, count = count, since='2018-10-16',until='2019-03-20',result_type='popular') 
  
            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 
                #tweeted_datetime = parser.parse(str(tweet.created_at))
                #print(tweeted_datetime)
                #print(type(tweeted_datetime))
                #if tweeted_datetime > Ten_days_ago:
                #print tweet, tweeted_datetime.strftime("%Y-%m-%d %H:%M:%S %p")
                # saving text of tweet 
                parsed_tweet['text'] = tweet.text 
                print(tweet.created_at)
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = get_tweet_sentiment(tweet.text) 
  
                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            # return parsed tweets 
            return tweets 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e))

In [10]:
def clean_tweet(tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 
tweet = 'RT @marcobonzanini: just an example! :D http://example.com #NLP'
print(clean_tweet(tweet))
  

RT just an example D NLP


In [ ]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 
tweet = 'RT @marcobonzanini: just an example! :D http://example.com #NLP'
print(preprocess(tweet))
  

In [ ]:
import operator 
import json
from collections import Counter
 
fname = 'mytweets.json'
with open(fname, 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        # Create a list with all the terms
        terms_all = [term for term in preprocess(tweet['text'])]
        # Update the counter
        count_all.update(terms_all)
    # Print the first 5 most frequent words
    print(count_all.most_common(5)

In [ ]:
from nltk.corpus import stopwords
import string
 
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']

In [ ]:
terms_stop = [term for term in preprocess(tweet['text']) if term not in stop]# Count terms only once, equivalent to Document Frequency
terms_single = set(terms_all)
# Count hashtags only
terms_hash = [term for term in preprocess(tweet['text']) 
              if term.startswith('#')]
# Count terms only (no hashtags, no mentions)
terms_only = [term for term in preprocess(tweet['text']) 
              if term not in stop and
              not term.startswith(('#', '@'))] 
              # mind the ((double brackets))
              # startswith() takes a tuple (not a list) if 
              # we pass a list of inputs

In [5]:
t=get_tweets("IPL",1000)
len(t)

NameError: name 'get_tweets' is not defined

In [ ]:
from tweepy import Stream
from tweepy.streaming import StreamListener
 
class MyListener(StreamListener):
 
    def on_data(self, data):
        try:
            with open('python.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True
 
twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=['#IPL'])